 # 4D-OBC merge
 
This notebook explains how to use the merge-function of the py4dgeo-packag which automatically merges similar 4D-OBCs to one.  

Implemented in: `py4dgeo.merge`

**Author(s)** 

Scharnagl, Robin (Technische Universität München)

**Related publication**

 Ulm M, Elias M,  Eltner A, Lotsari E & Anders K. (2025). Automated change detection in photogrammetric 4D point clouds – transferability and extension of 4D objects‑by‑change 
for monitoring riverbank dynamics using low‑cost cameras. *Applied Geomatics*, https://doi.org/10.1007/s12518-025-00623-9   


## **Method description**
This section should provide a detailed technical explanation of the method:

- **Theoretical foundation**: Due to uncorrelated noise, the extraction of 4D-OBCs tends to oversegment, which leads to multiple 4D-OBCs describing one process. This function merges 4D-OBCs that overlap in time and space, to create just one object.

- **Algorithm**: The algorithm first checks the change direction of each object. It then calculates the spatial and temporal overlap of the OBCs. If the OBC's changes point in the same direction, and their temporal and spatial overlap exceeds a user-defined threshold, their link is stored. In the last step, these get fused to one single 4D-OBC

- **Key parameters**: The parameters that the user can define are the following:
    - `time_threshold` (float, 0 - 1): threshold for temporal overlap, 1: complete overlap, 0: no overlap
    - `spatial_threshold`(float, 0 - 1): thershold for spatial overlap, 1: complete overlap, 0: no overlap


In [4]:
# Load Required Package
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import pooch
# Load Dataset

# Download data from zenodo and set path to point cloud folder
p = pooch.Pooch(base_url="doi:10.5281/zenodo.10003574/", path=pooch.os_cache("py4dgeo"))
p.load_registry_from_doi()
p.fetch("module3.zip", processor=pooch.Unzip(), progressbar=True)
pc_dir = p.path / "module3.zip.unzip" / "module3" / "schneeferner" / "pointclouds"


ReadTimeout: HTTPSConnectionPool(host='zenodo.org', port=443): Read timed out. (read timeout=30)

After loading the SpatiotemporalAnalysis-object, we can simply add the merge-function and define the parameters to the users-preference. 

In [ ]:
# Run the Method
merged= analysis.merge(time_threshold=0.7, spatial_threshold=0.1)

# Plot a merged object of change
merged[0].plot()

## References
-  Ulm M, Elias M,  Eltner A, Lotsari E & Anders K. (2025). Automated change detection in photogrammetric 4D point clouds – transferability and extension of 4D objects‑by‑change 
for monitoring riverbank dynamics using low‑cost cameras. *Applied Geomatics*, pages. https://doi.org/10.1007/s12518-025-00623-9   

## ✅ Contributor Checklist

- [ ] This notebook runs start to finish without errors
- [ ] Dataset is available via link to repository or DOI
- [ ] All method parameters are clearly explained
- [ ] The algorithm is thoroughly explained with appropriate theoretical background
- [ ] Results are visualized and properly interpreted (if applicable)
- [ ] All references are properly cited

Questions? Please refer to the contributor guide in the main `README.md` or open an issue on the GitHub issue tracker.
